In [1]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

!pip install antspyx
import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json

Gtk-Message: 17:18:56.648: Failed to load module "canberra-gtk-module"


In [2]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

17:19:12: Debug: Adding duplicate image handler for 'Windows bitmap file'
17:19:12: Debug: Adding duplicate animation handler for '1' type
17:19:12: Debug: Adding duplicate animation handler for '2' type
17:19:12: Debug: Adding duplicate image handler for 'Windows bitmap file'
17:19:12: Debug: Adding duplicate animation handler for '1' type
17:19:12: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:26287): Gtk-CRITICAL **: 17:19:12.767: gtk_window_resize: assertion 'height > 0' failed


In [3]:
import os.path as op
import os
import pandas as pd
from mne.datasets import sample
# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm

def mkdir_no_exist(path):
    if not op.isdir(path):
        os.makedirs(path)
        
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def direct_file_download_open_neuro(file_list, file_types, dataset_id, dataset_version, save_dirs):
    # https://openneuro.org/crn/datasets/ds004226/snapshots/1.0.0/files/sub-001:sub-001_scans.tsv
    for i, n in enumerate(file_list):
        subject = n.split('_')[0]
        download_link = 'https://openneuro.org/crn/datasets/{}/snapshots/{}/files/{}:{}:{}'.format(dataset_id, dataset_version, subject, file_types[i],n)
        print('Attempting download from ', download_link)
        download_url(download_link, op.join(save_dirs[i], n))
        print('Ok')

In [5]:
# Define the dataset and subject information
dataset_fmap = 'ds000171'
subject_fmap = '001'

# Set up the main directory for the dataset
bids_root = op.join(op.dirname(sample.data_path()), dataset_fmap)

# Define paths for functional and anatomical data
func_path = op.join(bids_root, 'sub-control01', 'func')
anat_path = op.join(bids_root, 'sub-control01', 'anat')
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')


# Create directories for func and anac
mkdir_no_exist(bids_root)
mkdir_no_exist(op.join(bids_root, 'sub-control01'))
mkdir_no_exist(func_path)
mkdir_no_exist(anat_path)
mkdir_no_exist(deriv_root)
mkdir_no_exist(preproc_root)

# List of functional files to download
file_list = [
    'sub-control01_task-music_run-1_bold.nii.gz',
    'sub-control01_task-music_run-1_events.tsv',
    'sub-control01_task-music_run-2_bold.nii.gz',
    'sub-control01_task-music_run-2_events.tsv',
    'sub-control01_task-music_run-3_bold.nii.gz',
    'sub-control01_task-music_run-3_events.tsv',
]

file_types = ['func'] * len(file_list)
save_dirs = [func_path] * len(file_list)

# Download functional files
direct_file_download_open_neuro(file_list=file_list, 
                                 file_types=file_types, 
                                 dataset_id=dataset_fmap, 
                                 dataset_version='00001', 
                                 save_dirs=save_dirs)



# List of anatomical files to download
anat_file_list = ['sub-control01_T1w.nii.gz']

# For the anac files, define their types and save directories
anat_file_types = ['anat'] * len(anat_file_list)  # Adjust as necessary
anat_save_dirs = [anat_path] * len(anat_file_list)

# Download anatomical files
direct_file_download_open_neuro(file_list=anat_file_list, 
                                 file_types=anat_file_types, 
                                 dataset_id=dataset_fmap, 
                                 dataset_version='00001', 
                                 save_dirs=anat_save_dirs)

Attempting download from  https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/sub-control01:func:sub-control01_task-music_run-1_bold.nii.gz


sub-control01:func:sub-control01_task-music_run-1_bold.nii.gz: 35.5MB [00:08, 4.06MB/s]                  


Ok
Attempting download from  https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/sub-control01:func:sub-control01_task-music_run-1_events.tsv


sub-control01:func:sub-control01_task-music_run-1_events.tsv: 8.19kB [00:00, 23.1kB/s]


Ok
Attempting download from  https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/sub-control01:func:sub-control01_task-music_run-2_bold.nii.gz


...

Ok
Attempting download from  https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/sub-control01:func:sub-control01_task-music_run-3_events.tsv


sub-control01:func:sub-control01_task-music_run-3_events.tsv: 8.19kB [00:00, 12.3kB/s]


Ok
Attempting download from  https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/sub-control01:anat:sub-control01_T1w.nii.gz


sub-control01:anat:sub-control01_T1w.nii.gz: 9.21MB [00:02, 3.64MB/s]                

Ok


In [7]:
import nibabel as nib
import numpy as np
from nilearn.image import concat_imgs

# Load the individual fMRI runs
run1 = nib.load(op.join(func_path, 'sub-control01_task-music_run-1_bold.nii.gz'))
run2 = nib.load(op.join(func_path, 'sub-control01_task-music_run-2_bold.nii.gz'))
run3 = nib.load(op.join(func_path, 'sub-control01_task-music_run-3_bold.nii.gz'))

concatenated_file_path = '/home/jovyan/mne_data/ds000171/sub-control01/func/concatenated_task_music_bold.nii.gz'

# Preprocess each run (example step: standardization)
def standardize_img(img):
    data = img.get_fdata()
    mean = np.mean(data)
    std = np.std(data)
    standardized_data = (data - mean) / std
    return nib.Nifti1Image(standardized_data, img.affine)

# Standardize each run
run1_standardized = standardize_img(run1)
run2_standardized = standardize_img(run2)
run3_standardized = standardize_img(run3)

# Concatenate the standardized runs into a single 4D image
concatenated_img = concat_imgs([run1_standardized, run2_standardized, run3_standardized])

# Save the concatenated image
nib.save(concatenated_img, concatenated_file_path)

print("Concatenation complete and saved as 'concatenated_task_music_bold.nii.gz'.")

Concatenation complete and saved as 'concatenated_task_music_bold.nii.gz'.


In [8]:
concatenated_file_path = '/home/jovyan/mne_data/ds000171/sub-control01/func/concatenated_task_music_bold.nii.gz'

fsleyesDisplay.resetOverlays()

if op.exists(concatenated_file_path):
    fsleyesDisplay.load(concatenated_file_path)
    
    print("Loaded:", concatenated_file_path)
else:
    print("File does not exist at the specified path.")


(ipykernel_launcher.py:26287): Gdk-WARNING **: 17:37:34.924: gdkdrawable-x11.c:952 drawable is not a pixmap or window


Loaded: /home/jovyan/mne_data/ds000171/sub-control01/func/concatenated_task_music_bold.nii.gz


In [10]:
otherfiles_path = bids_root  # Assuming this is the desired location

otherfiles_list = [
    'CHANGES',
    'README',
    'dataset_description.json',
    'T1w.json',
    'participants.tsv',
    'task-music_bold.json',
    'task-nonmusic_bold.json'
]

# Adjust the function to handle these files
for filename in otherfiles_list:
    # Construct the download link for these files
    download_link = f'https://openneuro.org/crn/datasets/{dataset_fmap}/snapshots/00001/files/{filename}'
    print(f'Attempting download from: {download_link}')
    
    # Download the file without specifying file_type
    download_url(download_link, op.join(otherfiles_path, filename))
    print('Downloaded:', filename)

Attempting download from: https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/CHANGES


CHANGES: 8.19kB [00:00, 14.7kB/s]


Downloaded: CHANGES
Attempting download from: https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/README


README: 8.19kB [00:00, 13.5kB/s]


Downloaded: README
Attempting download from: https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/dataset_description.json


...

Downloaded: participants.tsv
Attempting download from: https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/task-music_bold.json


task-music_bold.json: 8.19kB [00:00, 15.2kB/s]


Downloaded: task-music_bold.json
Attempting download from: https://openneuro.org/crn/datasets/ds000171/snapshots/00001/files/task-nonmusic_bold.json


task-nonmusic_bold.json: 8.19kB [00:00, 14.5kB/s]

Downloaded: task-nonmusic_bold.json


In [11]:
print_dir_tree(bids_root, max_depth=4)

|ds000171/
|--- CHANGES
|--- README
|--- T1w.json
|--- dataset_description.json
|--- participants.tsv
|--- task-music_bold.json
|--- task-nonmusic_bold.json
|--- derivatives/
|------ preprocessed_data/
|--- sub-control01/
|------ anac/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- concatenated_task_music_bold.nii.gz
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii.gz
|--------- sub-control01_task-music_run-2_events.tsv
|--------- sub-control01_task-music_run-3_bold.nii.gz
|--------- sub-control01_task-music_run-3_events.tsv


In [13]:
def get_skull_stripped_anatomical(bids_root, preproc_root, subject_id, robust=False):
    """
    Perform skull-stripping on the T1w anatomical image for the specified subject.
        Parameters
    ----------
    bids_root: string
        The root of the BIDS directory
    preproc_root: string
        The root of the preprocessed data, where the result of the brain extraction will be saved.
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """
    # Correct subject ID for the dataset
    subject = f'sub-{subject_id}'  # Ensure this matches your dataset structure
    anatomical_path = op.join(bids_root, subject, 'anat', f'{subject}_T1w.nii.gz')
    
    # Preprocessing output path
    betted_brain_path = op.join(preproc_root, subject, 'anat', f'{subject}_T1w')
    
    # Create the directory for preprocessed data if it doesn't exist
    mkdir_no_exist(op.join(preproc_root, subject, 'anat'))
    
    # Run BET
    os.system(f'bet {anatomical_path} {betted_brain_path} -m {"-R" if robust else ""}')
    print("Done with BET.")


resulting_mask_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_mask')
# Update the call to match your dataset's subject ID
get_skull_stripped_anatomical(bids_root, preproc_root, "control01")


Done with BET.


In [14]:
fsleyesDisplay.load(resulting_mask_path)

In [16]:
get_skull_stripped_anatomical(bids_root, preproc_root, "control01", robust=True)

Done with BET.


In [19]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(concatenated_file_path)
fsleyesDisplay.load(resulting_mask_path)

In [20]:
def apply_fsl_math_approach(img_path, mask_path, masked_img_path):
    """
    Apply FSL's fslmaths to mask an image with a given mask.

    Parameters
    ----------
    img_path: str
        Path to the input image that needs to be masked.
    mask_path: str
        Path to the mask image.
    masked_img_path: str
        Path where the masked output image will be saved.
    """
    # Execute the fslmaths command with the -mas option
    os.system('fslmaths {} -mas {} {}'.format(img_path, mask_path, masked_img_path))
    print(f"Masked image saved at: {masked_img_path}")

# Define paths based on your dataset structure
subject_id = "control01"  # Adjust this as necessary
anatomical_path = op.join(bids_root, f'sub-{subject_id}', 'anat', f'sub-{subject_id}_T1w.nii.gz')  # Original brain
betted_brain_path = op.join(preproc_root, f'sub-{subject_id}', 'anat', f'sub-{subject_id}_T1w.nii.gz')  # Skull-stripped brain
resulting_mask_path = op.join(preproc_root, f'sub-{subject_id}', 'anat', f'sub-{subject_id}_T1w_mask.nii.gz')  # The mask

# Apply the FSL math approach
apply_fsl_math_approach(betted_brain_path, resulting_mask_path, anatomical_path)


Masked image saved at: /home/jovyan/mne_data/ds000171/sub-control01/anat/sub-control01_T1w.nii.gz


In [21]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

In [23]:
import glob
from nipype.interfaces.fsl import FAST
subject = f'sub-{subject_id}'
anatomical_path = op.join(bids_root, subject, 'anat', f'{subject}_T1w.nii.gz')
bet_path = op.join(preproc_root, subject, 'anat', f'{subject}_T1w')

##########################
fast_target = bet_path # Replace with either anatomical_path or bet_path (note: you can try both and decide which is more reasonable!)

[os.remove(f) for f in glob.glob(op.join(preproc_root, subject, 'anat', '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(preproc_root, subject, 'anat', f'{subject}_T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)
print("FAST segmentation complete.")

FAST segmentation complete.


In [24]:
print_dir_tree(bids_root, max_depth=5)

|ds000171/
|--- CHANGES
|--- README
|--- T1w.json
|--- dataset_description.json
|--- participants.tsv
|--- task-music_bold.json
|--- task-nonmusic_bold.json
|--- derivatives/
|------ preprocessed_data/
|--------- sub-001/
|------------ anat/
|--------- sub-control01/
|------------ anat/
|--------------- sub-control01_T1w.nii.gz
|--------------- sub-control01_T1w_fast_mixeltype.nii.gz
|--------------- sub-control01_T1w_fast_pve_0.nii.gz
|--------------- sub-control01_T1w_fast_pve_1.nii.gz
|--------------- sub-control01_T1w_fast_pve_2.nii.gz
|--------------- sub-control01_T1w_fast_pveseg.nii.gz
|--------------- sub-control01_T1w_fast_seg.nii.gz
|--------------- sub-control01_T1w_mask.nii.gz
|--- sub-control01/
|------ anac/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- concatenated_task_music_bold.nii.gz
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii

In [27]:
import subprocess
dataset_demo = 'ds000171'
subject_demo = '001'

# Define paths
sample_path = "/home/jovyan/Data/dataset"
mkdir_no_exist(sample_path)

# Set BIDS and preprocessed root directories
bids_root_demo = op.join(os.path.abspath(""), sample_path, dataset_demo)
preproc_root_demo = op.join(bids_root_demo, 'derivatives')

# Create the necessary directories
mkdir_no_exist(bids_root_demo)
mkdir_no_exist(preproc_root_demo)

# Download the dataset
subprocess.run([
    "openneuro-py",
    "download",
    "--dataset", dataset_demo,
    "--target-dir", bids_root_demo,
    "--include", f'sub-{subject_demo}'
], check=True)


👋 Hello! This is openneuro-py 2024.2.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds000171 …
Cannot determine local revision of the dataset, and the target directory is not empty. If the download fails, you may want to try again with a fresh (empty) target directory.


📁 Traversing directories for ds000171 : 7 entities [00:08,  1.25s/ entities]
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /opt/conda/lib/python3.11/site-packages/openneuro/_cli.py:64 in download_cli │
│                                                                              │
│    61 │   ] = 5,                                                             │
│    62 ) -> None:                                                             │
│    63 │   """Download datasets from OpenNeuro."""                            │
│ ❱  64 │   download(                                                          │
│    65 │   │   dataset=dataset,                                               │
│    66 │   │   tag=tag,                                                       │
│    67 │   │   target_dir=target_dir,                                         │
│                                                                              │
│ ╭──────────────────────────── 

CalledProcessError: Command '['openneuro-py', 'download', '--dataset', 'ds000171', '--target-dir', '/home/jovyan/Data/dataset/ds000171', '--include', 'sub-001']' returned non-zero exit status 1.

In [ ]:
import subprocess
dataset_demo = 'ds000114'
subject_demo = '01'

sample_path = "/home/jovyan/Data/dataset"
mkdir_no_exist(sample_path)

# Download one subject's data from each dataset
bids_root_demo = op.join(os.path.abspath(""),sample_path, dataset_demo)
preproc_root_demo = op.join(bids_root_demo, 'derivatives')

subprocess.run(["openneuro-py", "download", "--dataset", dataset_demo, "--target-dir", bids_root_demo, "--include", 'sub-{}'.format(subject_demo)], check=True)

In [ ]:
file_to_trim = glob.glob(op.join(bids_root_demo, 'sub-01', 'ses-test', 'func', 'sub-01_ses-test_task-fingerfootlips_bold.nii.gz'))[0]
mkdir_no_exist(preproc_root_demo)
mkdir_no_exist(op.join(preproc_root_demo, 'sub-01'))
mkdir_no_exist(op.join(preproc_root_demo, 'sub-01', 'ses-test'))
mkdir_no_exist(op.join(preproc_root_demo, 'sub-01', 'ses-test','func'))
output_target = op.join(preproc_root_demo, 'sub-01', 'ses-test', 'func', 'sub-01_ses-test_task-fingerfootlips_bold_settled.nii.gz')

############################
# Solution
# We will start from the 10th volume.
# Because we start at 0, we begin at the 11th volume, excluding 10 volumes: we should start at number 10.
# There are 184 volumes. We remove 11 volumes, so we're left with 174 volumes.
###########################

# For this, knowing that there are originally 184 volumes and that you want to throw away the first 10, please fill in
# the following variables
start_vol = 10 # Where should we start? (First volume is 0, not 1 !)
number_of_volumes = 174 # How many volumes should we keep?

fslroi(file_to_trim, output_target, str(start_vol), str(number_of_volumes))

In [ ]:
import os
import os.path as op
import glob
from nipype.interfaces.fsl import FAST

def run_fast_on_brain_extracted_image(bids_root, preproc_root, subject_id):
    
    # Define paths based on your dataset structure
    subject = f'sub-{subject_id}'  # For example: sub-001
    anatomical_path = op.join(bids_root, subject, 'anat', f'{subject}_T1w.nii.gz')  # Original T1-weighted image
    bet_path = op.join(preproc_root, subject, 'anat', f'{subject}_T1w')  # Skull-stripped brain

    # Define the output path for FAST segmentation
    segmentation_path = op.join(preproc_root, subject, 'anat', f'{subject}_T1w_fast')

    # Clean up any previous FAST output files in the directory
    [os.remove(f) for f in glob.glob(op.join(preproc_root, subject, 'anat', '*fast*'))]

    # Run FAST on the brain-extracted image
    fast = FAST()
    fast.inputs.in_files = bet_path  # Use the BET output as input
    fast.inputs.out_basename = segmentation_path  # Output basename for FAST
    fast.inputs.n_classes = 3  # Number of tissue classes to segment
    fast.run()  # Execute the segmentation

    print("FAST segmentation complete.")

# Example call
run_fast_on_brain_extracted_image(bids_root, preproc_root, "001")  # Replace "001" with the correct subject ID
